# DATASET LINK

In [ ]:
# Link al dataset de Kaggle usado en el proyecto:
"https://www.kaggle.com/datasets/tusharbhadouria/credit-card-fraud-detection"

# INSTALL PANDAS AND NUMPY

In [ ]:
# Importamos las librerías necesarias para el análisis de datos, manipulación y visualización: pandas y numpy.
import pandas as pd
import numpy as np

# LOAD DATASETS

In [ ]:
# Cargamos los datos de entrenamiento y test desde archivos CSV. Como los datasets son muy grandes y tardan en cargarse, utilizamos una librería para paralelizar el código de carga.
# Librería utilizada: multiprocessing
from multiprocessing.pool import ThreadPool

# Función para cargar los archivos csv que tenemos como datasets.
def carregar(nom):
    return pd.read_csv(nom)

# Los csv que cargaremos son los siguientes:
csvs = ['fraudTrain.csv', 'fraudTest.csv']

# Creamos 2 grupos porque tenemoso dos archivos csv.
pool = ThreadPool(2)

# Cargamos los archivos en paralelo.
resultats = pool.map(carregar, csvs)

# Guardamos los resultados en sus respectivas variables de dataset de entrenamiento y test.
train_data = resultats[0]
test_data = resultats[1]


In [ ]:
# Visualizamos las primeras filas del conjunto de datos de entrenamiento para entender su estructura.
train_data.head()

## DATA PREPROCESSING

In [ ]:
# Mostramos las dimensiones de los conjuntos de datos de entrenamiento y test, para verificar la cantidad de muestras y características.
print(f'{train_data.shape}')
print(f'{test_data.shape}')

In [ ]:
# Obtenemos información detallada sobre el conjunto de datos de entrenamiento, incluyendo tipos de datos y valores nulos, para evaluar la calidad de los datos y saber si necesitamos realizar limpieza o preprocesamiento adicional..
train_data.info()

#### NO NULL VALUES
Podemos ver que no hay valores nulos en ninguna columna. No hay nulls en todo el dataset, por lo tanto, todos los datos son válidos en este aspecto y no nos tenemos que preocupar por ello. No obstante, ahora tenemos que procesar los datos: transformar objetos y strings a valors numéricos. Además, columnas de IDs como cc_num y trans_num se pueden eliminar del dataset porque los valores de identidad no nos van a aportar ningún tipo de información de valor en este proyecto.


### TRANSFORM DATES TO TIMESTAMP

In [ ]:
# Primero, eliminamos la primera columna del dataset, que es un índice y no aporta ningún tipo de información relevante para el análisis.
train_data = train_data.drop(columns='Unnamed: 0')
test_data = test_data.drop(columns='Unnamed: 0')


In [ ]:
# Ahora, convertimos las fechas a números, específicamente a timestamps en segundos desde la época Unix (1 de enero de 1970).
train_data['trans_date_trans_time'] = pd.to_datetime(train_data['trans_date_trans_time']).astype('int64') // 10**9
test_data['trans_date_trans_time'] = pd.to_datetime(test_data['trans_date_trans_time']).astype('int64') // 10**9

# Convertimos la columna 'dob' (fecha de nacimiento) a timestamps en segundos.
train_data['dob'] = pd.to_datetime(train_data['dob']).astype('int64') // 10**9
test_data['dob'] = pd.to_datetime(test_data['dob']).astype('int64') // 10**9

In [ ]:
# Visualizamos las primeras filas del conjunto de datos de entrenamiento para verificar los cambios realizados.
train_data.head(1)


In [ ]:
# Visualizamos las primeras filas del conjunto de datos de test para verificar los cambios realizados.
test_data.head(1)

#### AÚN FALTA TRANSFORMAR ALGUNAS COLUMNAS A VALORES NÚMEROS, Y POR ELLO UTILIZAMOS UN ENCODER

In [ ]:
# Aquí, utilizamos OrdinalEncoder de sklearn para convertir las columnas categóricas en números, facilitando así su uso en modelos de machine learning.
# Para ello, importamos sklearn.preprocessing.OrdinalEncoder.
from sklearn.preprocessing import OrdinalEncoder

# Eliminamos las columnas de IDs que no aportan valor predictivo.
train_data = train_data.drop(columns=['cc_num', 'trans_num'])
test_data = test_data.drop(columns=['cc_num', 'trans_num'])

# Lista de las columnas a codificar con el OrdinalEncoder.
categorical_cols = ['gender', 'category', 'merchant', 'job', 'city', 'state', 'street', 'first', 'last']

# Creamos un OrdinalEncoder que se encargue de valores desconocidos.
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)

# Entrenamos el Encoder con los datos de entrenamiento (paso previo necesario para la paralelización).
encoder.fit(train_data[categorical_cols])

# Función para aplicar la transformación en paralelo.
def aplicar_encoder(df):
    # Entrenamos el Encoder en los datos de entrenamiento y los transformamos (asigna el número por orden alfabético):
    # Transformamos los datos del test usando el encoder entrenado, y los valores desconocidos que no ha visto en el entrenamiento se codifican con un -1.
    df[categorical_cols] = encoder.transform(df[categorical_cols])
    return df

# Creamos un grupo de 2 hilos para procesar train y test simultáneamente.
pool_enc = ThreadPool(2)
resultats_list = pool_enc.map(aplicar_encoder, [train_data, test_data])

# Recuperamos los resultados procesados.
train_data, test_data = resultats_list[0], resultats_list[1]

# Visualizamos los resultados codificados
train_data.head(1)

# TRAIN SIMPLE MODELS TO HAVE AN IDEA OF % OF ACCURACY, RECALL AND F1-SCORE

In [ ]:
# Separamos las características (X_) de la variable objetivo (y_) del dataset de entrenamiento.
X_train = train_data.drop(columns='is_fraud')
y_train = train_data['is_fraud']

# Hacemos lo mismo para el dataset de test.
X_test = test_data.drop(columns='is_fraud')
y_test = test_data['is_fraud']  

In [ ]:
# Importamos RandomForestClassifier de sklearn para entrenar un modelo de clasificación.
import sklearn
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# Entrenamos un modelo de Random Forest con los datos de entrenamiento y los parámetros especificados.
# Importamos os para poder contar el número de cpus que tenemos, y vamos a usar un porcentaje de ellas, un 75%, que trabajen de forma paralela cuando entrenamos el modelo del Random Forest.
# Paralelizando así el código se ejecuta más rápido, pero dejamos un 25% de cpus disponibles para otras tareas del ordenador y que este no se cuelgue y pare.
import os
n_cores_mlp = os.cpu_count()
safe_cores_rf = max(1, int(n_cores_mlp * 0.75))
model = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=2003, n_jobs=safe_cores_rf)
model.fit(X_train, y_train)

In [ ]:
# Hacemos las predicciones en el conjunto de test.
y_pred = model.predict(X_test)

In [ ]:
# Importamos las métricas necesarias para evaluar el rendimiento del modelo, concretamente accuracy, recall y f1-score.
from sklearn.metrics import accuracy_score, recall_score
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1score}')

In [ ]:
# Importamos Optuna y los módulos necesarios para la optimización de hiperparámetros. Aquí obtendremos los mejores parámetros para maximizar el recall (detección de fraudes) del Random Forest.
import optuna
from optuna.pruners import MedianPruner # MedianPruner para detener ensayos que no muestran mejora.
from optuna.samplers import TPESampler # TPE Sampler para una búsqueda eficiente de hiperparámetros. Lo que hace es una optimización bayesiana para seleccionar los mejores hiperparámetros basándose en los resultados de ensayos anteriores.

# Definimos la función objetivo que Optuna utilizará para evaluar diferentes combinaciones de hiperparámetros. En ella, entrenamos un RandomForestClassifier con los parámetros de prueba y calculamos el recall en el conjunto de test.
# Luego, Optuna intentará maximizar este valor, buscando los mejores hiperparámetros y mejorando así la capacidad del modelo para detectar fraudes.

def objective(trial):
    # Definimos los hiperparámetros a optimizar.
    n_estimators = trial.suggest_int('n_estimators', 20, 50) # Número de árboles en el bosque.
    
    # Ahora, debemos ver si max_depth es None o un valor específico.
    max_depth_choice = trial.suggest_int('max_depth_choice', 0, 1) # Si es 0, max_depth será un valor entre 10 y 50; si es 1, será None. Esto se deber hacer así porque Optuna no permite sugerir None directamente.
    max_depth = None if max_depth_choice == 1 else trial.suggest_int('max_depth', 10, 50) 
    
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10) # Mínimo número de muestras necesarias para dividir un nodo.
    
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5) # Mínimo número de muestras necesarias en una hoja.
    
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None]) # Número de características a considerar al buscar la mejor división. 
    # 'sqrt' usa la raíz cuadrada del número total de características, 'log2' usa el logaritmo base 2 y None usa todas las características. 
    
    
    
    # Creamos el Random Forest con los hiperparámetros sugeridos.
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=2003
    )
    
    # Entrenamos el modelo con los datos de entrenamiento.
    rf.fit(X_train, y_train)
    
    # Hacemos las predicciones en el conjunto de test.
    y_pred = rf.predict(X_test)
    
    # Calculamos el recall (nuestro objetivo de optimización).
    recall = recall_score(y_test, y_pred)
    
    return recall

# Creamos un estudio de Optuna para maximizar el recall, utilizando el TPE Sampler para una búsqueda eficiente y el Median Pruner para detener ensayos que no muestran mejora.
# Luego usaremos este estudio para optimizar los hiperparámetros del Random Forest.
study = optuna.create_study(
    direction='maximize', # direction='maximize' porque queremos maximizar el recall.
    sampler=TPESampler(seed=2003), # Random State fijado para reproducibilidad.
    pruner=MedianPruner() # Usamos MedianPruner para detener ensayos que no muestran mejora.
)

# Ahora, optimizamos los hiperparámetros ejecutando múltiples ensayos. 
# Aquí lo que hacemos es ejecutar la función objetivo 50 veces, cada vez con una combinación diferente de hiperparámetros sugerida por Optuna, para encontrar la que maximiza el recall.
study.optimize(objective, n_trials=50, show_progress_bar=True, n_jobs=safe_cores_rf)

# Guardamos los mejores resultados obtenidos tras la optimización de hiperparámetros.
best_trial = study.best_trial
print(f"\nBest trial:")
print(f"  Recall: {best_trial.value:.4f}")
print(f"  Parameters: {best_trial.params}")

# Entrenamos el modelo final con los mejores parámetros encontrados por Optuna.
best_params = best_trial.params
# Ajustamos max_depth según la elección hecha durante la optimización.
max_depth_final = None if best_params['max_depth_choice'] == 1 else best_params.get('max_depth')

# Finalmente, definimos el mejor modelo con los mejores hiperparámetros encontrados.
best_model = RandomForestClassifier(n_estimators=best_params['n_estimators'], max_depth=max_depth_final, min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'], max_features=best_params['max_features'], random_state=2003)

# Entrenamos el mejor modelo con los datos de entrenamiento.
best_model.fit(X_train, y_train)

# Hacemos las predicciones en el conjunto de test con el mejor modelo.
y_pred_best = best_model.predict(X_test)

# Por último, calculamos y mostramos las métricas de rendimiento del mejor modelo en el conjunto de test.
accuracy_best = accuracy_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = sklearn.metrics.f1_score(y_test, y_pred_best)

print(f"\nBest Model Performance on Test Set:")
print(f'Accuracy: {accuracy_best:.4f}')
print(f'Recall: {recall_best:.4f}')
print(f'F1-Score: {f1_best:.4f}')

## MLP

In [ ]:
# En esta sección vamos a optimizar los hiperparámetros de un MLPClassifier. Para ello, importamos las librerías siguientes, brevemente explicadas:
# - optuna: Librería para optimización de hiperparámetros.
# - sklearn.metrics: Módulo para evaluar el rendimiento del modelo.
# - sklearn.model_selection: Módulo para validación cruzada.
# - sklearn.neural_network: Módulo que contiene el clasificador MLP.
# - sklearn.preprocessing: Módulo para escalado de características.
# - imblearn.over_sampling: Módulo para aplicar SMOTE.
# - imblearn.pipeline: Módulo para crear pipelines que incluyen SMOTE.
# - import os, psutil: Módulos para gestionar recursos del sistema, ya que esta optimización a veces puede consumir muchos recursos.
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import os
import psutil

# Necesitamos realizar este paso porque el entrenamiento de MLP puede ser intensivo en recursos, y esto hace que nuestro ordenador se cuelgue si no lo controlamos bien.
# En este paso, miramos cuantos cpus tenemos disponibles y vamos a usar un porcentaje de ellos, no todos porque podemos hacer que nuestro 
# ordenador se cuelgue
n_cores_mlp = os.cpu_count()

print(f"Available CPU cores: {n_cores_mlp}")

# Aquí determinamos que vamos a usar el 75% de los cores disponibles en nuestro ordenador.
safe_cores_mlp = max(1, int(n_cores_mlp * 0.75))


# Definimos la función de optimización con Optuna para el MLP. Evaluaremos diferentes combinaciones de hiperparámetros y entrenaremos distintos MLP con esos parámetros. Entonces,
# Optuna intentará buscar los mejores hiperparámetros, tal y como hemos hecho en el Random Forest anteriormente. 

def objective_mlp(trial, X, y):
    # Definimos los hiperparámetros a optimizar para el MLP.
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50, 50), (100, 50)]) # Provamos distintas arquitecturas de capas del MLP.
    activation = trial.suggest_categorical('activation', ['relu', 'tanh']) # Provamos distintas activaciones del MLP.
    solver = trial.suggest_categorical('solver', ['adam']) # Como solver usamos adam porque sgd a veces es demasiado lento.
    alpha = trial.suggest_float('alpha', 1e-5, 1e-2, log=True) # El parámetro alpha, que ayuda a prevenir el overfitting si el modelo es complejo.
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True) # El learning rate controla la velocidad del ajuste de pesos.
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256]) # El batch size es el número de muestras que procesa el modelo antes de actualizar sus pesos.
    
    # Creamos el MLP con los valores que acabos de definir anteriormente, para más adelante encontrar los valores más óptimos.
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        alpha=alpha,
        learning_rate_init=learning_rate_init,
        batch_size=batch_size,
        max_iter=200, # Lo mantenemos bajo para la velocidad. 
        random_state=2003,
        early_stopping=True, # Usamos el early stopping para más velocidad y no perder tiempo si el modelo se estanca. 
        validation_fraction=0.1, # Fracción de los datos de entrenamiento que se utiliza para validar el modelo. 
        n_iter_no_change=10, # Iteraciones que puede hacer el modelo antes de detenerse si no hay mejora.
        verbose=0
    )
    
    # Instanciamos SMOTE porque nuestro dataset es muy desequilibrado, y con SMOTE creamos datos "sintéticos" para equilibrarlo. Es decir, hacemos oversampling de la clase minoritaria.
    smote = SMOTE(random_state=2003, k_neighbors=3)
    
    # Instanciamos el StandardScaler para escalar nuestros datos. 
    scaler = StandardScaler()
    
    # Creamos la Pipeline para seguir el orden correcto de escalar, aplicar el smote y entrenar el MLP.
    pipeline = Pipeline([
        ('scaler', scaler), 
        ('smote', smote),
        ('mlp', mlp)
    ])
    
    # Usando el número de cores especificados, calculamos la métrica del recall con cross_val_score. 
    cv_cores = max(1, safe_cores_mlp // 2)
    scores = cross_val_score(pipeline, X, y, scoring='recall', cv=3, n_jobs=cv_cores)
    recall_mean = scores.mean()
    

    
    return recall_mean

# Creamos un objeto de estudio para poder maximizar el recall, tal y como hemos hecho antes en el Random Forest, con parámetros parecidos. 
study_mlp = optuna.create_study(
    direction='maximize', # direction='maximize' porque queremos maximizar el recall.
    sampler=TPESampler(seed=2003), # Para reproducibilidad
    pruner=MedianPruner() # Usamos MedianPruner para detener ensayos que no muestran mejora.
)


# Para el estudio, usamos un total de cores que sea seguro para nuestro sistema, tal y como hemos hecho antes.
parallel_trials_mlp = max(1, safe_cores_mlp // 2)

# Ejecutamos la optimización con los parámetros que hemos definido ya.
study_mlp.optimize(
    lambda trial: objective_mlp(trial, X_train, y_train),
    n_trials=20,
    show_progress_bar=True,
    n_jobs=parallel_trials_mlp
)

# Cogemos el mejor estudio que ha obtenido la mejor solución. Con los parámetros ya especificados, vemos cuál es el mejor recall obtenido. 
best_trial_mlp = study_mlp.best_trial
print(f"\n\nBest MLP trial from cross-validation:")
print(f"  Mean CV Recall: {best_trial_mlp.value:.4f}") # Mejor recall medio.
print(f"  Parameters: {best_trial_mlp.params}") # Mejores parámetros.


# Ahora, entrenamos el MLP con los mejores parámetros que hemos obtenido.

# Primero de todo, escalamos los datos con el StandardScaler tanto los de entrenamiento como de test. 
scaler_final = StandardScaler()
X_train_scaled = scaler_final.fit_transform(X_train)
X_test_scaled = scaler_final.transform(X_test) 

# Luego, aplicamos el SMOTE porque el dataset está bastante desbalanceado. De esta manera, tenemos más fraudes para que el modelo los pueda analizar. 
smote_final_mlp = SMOTE(random_state=2003, k_neighbors=3)
X_train_smote_mlp, y_train_smote_mlp = smote_final_mlp.fit_resample(X_train_scaled, y_train) # Dataset con SMOTE aplicado ya.


# Evidentemente, el MLP que vamos a entrenar va a tener los mejores hiperparámetros que ya hemos encontrado antes.
best_params_mlp = best_trial_mlp.params

# Creamos el MLP con los parámetros.
best_mlp_model = MLPClassifier(
    hidden_layer_sizes=best_params_mlp['hidden_layer_sizes'],
    activation=best_params_mlp['activation'],
    solver=best_params_mlp['solver'],
    alpha=best_params_mlp['alpha'],
    learning_rate_init=best_params_mlp['learning_rate_init'],
    batch_size=best_params_mlp['batch_size'],
    max_iter=500,
    random_state=2003,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=10,
    verbose=0
)

# Entrenamos el modelo con el dataset con SMOTE aplicado.
best_mlp_model.fit(X_train_smote_mlp, y_train_smote_mlp)

# Hacemos las predicciones una vez entrenado el modelo sobre el dataset de X_test_scaled, este sin SMOTE porque es el de test.
y_pred_mlp = best_mlp_model.predict(X_test_scaled)

# A partir de las predicciones, ahora podemos calcular las distintas métricas que nos interesan y que ya hemos visto antes.
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print(f"\nFinal MLP Model Performance on Test Set:")
print(f'  Accuracy:  {accuracy_mlp:.4f}')
print(f'  Recall:    {recall_mlp:.4f}')
print(f'  F1-Score:  {f1_mlp:.4f}')

